In [2]:
import psycopg2 as pg
import json
import sys
import pandas.io.sql as psql
import plotly.plotly as py 

import numpy

try:
    conn = pg.connect(host='localhost')
except:
    print("I am unable to connect to the database")
    
cur = conn.cursor()
cur.execute("SET search_path = carvana")
    

In [3]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
import plotly.graph_objs as go

init_notebook_mode(connected=True)

import plotly
plotly.tools.set_credentials_file(username='adervish', api_key='Ddss5qpHA8iN2IWuqDJs')

In [8]:
dataframe = psql.read_sql(""" SELECT k.year,
    k.model,
    k.make,
    k.miles AS k_miles,
    (c.miles / 1000::numeric)::integer AS c_miles,
    k.miles - (c.miles / 1000::numeric)::integer::numeric AS miles_diff,
    k.price AS k_price,
    c.price AS c_price,
    k.price::text::double precision - c.price::text::double precision AS price_diff,
    k.descr AS k_trim,
    k.drivetrain::text AS k_drivetrain,
    c."trim" AS c_trim
   FROM kmx_vehicles k,
    carvana_vehicles c
  WHERE k.model = '"Civic"' 
      and k.year = c.year 
      AND k.model = c.model 
      AND k.make = c.make 
      AND k.make = '"Honda"'::jsonb AND abs(k.miles - (c.miles / 1000::numeric)::integer::numeric) < 3::numeric 
      AND k.descr::text ~~* (('%'::text || btrim(c."trim"::text, '"'::text)) || '"'::text) AND (k.drivetrain::text = '"2WD"'::text OR c.morefeatures ~~ (('%'::text || btrim(k.drivetrain::text, '"'::text)) || '%'::text))
  ORDER BY k.make, k.model, k.year""", conn)

In [10]:
data = [go.Histogram(x=dataframe['price_diff'])]
py.iplot(data, filename='honda civic price difference')